*A notebook comparing eigenstrat pca estimates to sklearn ...*

# Imports

In [ ]:
import numpy as np
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds
from sklearn.utils.extmath import svd_flip

import pandas as pd

import pcshrink

# Data prep

In [ ]:
%%time
data = pcshrink.UnpackedAncestryMap("/project/jnovembre/jhmarcus/ancient-sardinia/data/ref_genotypes/7-11-2018/lazaridis_2014/data_fil")

In [ ]:
print(data.n, data.p)

Find snps that are too rare or too common

In [ ]:
# use allele frequency estimator from Price et al. 2006
f = (1. + np.nansum(data.Y, axis=1)) / (2 + (2. * data.n))

# keep snps that aren't too rare or common
snp_idx = np.where((f > .02) & (f < .98))[0]
print(f.shape, snp_idx.shape)

Mean center and scale and impute missing values to 0

In [ ]:
Z = data.Y[snp_idx, :]

# mean genotype 
mu = np.nanmean(Z, axis=1).reshape(len(snp_idx), 1)

# empirical std deviation
std = np.nanstd(Z, axis=1).reshape(len(snp_idx), 1)

# heterozygosity scaler
het = np.sqrt(2. * f[snp_idx] * (1. - f[snp_idx])).reshape(len(snp_idx), 1)

Z = (Z - mu) / het
Z[np.isnan(Z)] = 0.0

In [ ]:
plt.scatter(het, std)
plt.xlabel("Het Scaler")
plt.ylabel("Emprical Std")

## Rewrite eigenstrat files at filtered SNPs

Save geno

In [ ]:
#Y = data.Y[snp_idx,:]
#Y[np.isnan(Y)] = 9.0
#Y = Y.astype(np.int8)
#Y

In [ ]:
#%%time
#np.savetxt("data/freq_fil/data_fil_freqfil.geno", Y, fmt="%.0f", delimiter="")

Save snp

In [ ]:
#snp_df = pd.read_table("data/data_fil.snp", header=None, sep="\t")[0].str.split(expand=True)
#snp_df = snp_df.iloc[snp_idx,:]
#snp_df.to_csv("data/freq_fil/data_fil_freqfil.snp", header=None, index=None, sep="\t")

Save ind

In [ ]:
#%%bash
#cp data/data_fil.ind data/freq_fil/data_fil_freqfil.ind

---

# PCA

In [ ]:
%%time
U, Sigma, VT = svds(Z.T, k=10)
Sigma = np.diag(Sigma[::-1])
U, VT = svd_flip(U[:, ::-1], VT[::-1])

L = U 
F = (Sigma @ VT).T

L = L / np.linalg.norm(L, axis=0, ord=2)
F = F / np.linalg.norm(F, axis=0, ord=2)

Plot PC1 vs PC2

In [ ]:
plt.scatter(L[:, 0], -L[:, 1])
plt.xlabel("PC1")
plt.ylabel("PC2")

Read eigenvectors from output from eigenstrat run with these params ...

In [ ]:
%%bash
cat data/freq_fil/pca.par

In [ ]:
pc_df = pd.read_table("./data/freq_fil/data_fil_freqfil.evec", header=None, sep="\t", skiprows=1)[0].str.lstrip().str.split(expand=True)

Convert pc_df to matrix

In [ ]:
L_eigen = pc_df.iloc[:,1:21].astype(float).as_matrix()

In [ ]:
plt.scatter(L_eigen[:,0], -L_eigen[:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")

Compare sklearn to eigenstrat ...

In [ ]:
plt.scatter(L[:,0], L_eigen[:,0])
plt.xlabel("PC1 (sklearn)")
plt.ylabel("PC1 (eigenstrat)")

In [ ]:
plt.scatter(L[:,1], L_eigen[:,1])
plt.xlabel("PC2 (scipy)")
plt.ylabel("PC2 (eigenstrat)")

In [ ]:
plt.scatter(L[:,2], L_eigen[:,2])
plt.xlabel("PC3 (scipy)")
plt.ylabel("PC3 (eigenstrat)")

In [ ]:
plt.scatter(L[:,3], L_eigen[:,3])
plt.xlabel("PC4 (scipy)")
plt.ylabel("PC4 (eigenstrat)")

In [ ]:
plt.scatter(L[:,4], -L_eigen[:,4])
plt.xlabel("PC5 (scipy)")
plt.ylabel("PC5 (eigenstrat)")

In [ ]:
plt.scatter(L[:,5], -L_eigen[:,5])
plt.xlabel("PC6 (scipy)")
plt.ylabel("PC6 (eigenstrat)")

In [ ]:
plt.scatter(L[:,6], -L_eigen[:,6])
plt.xlabel("PC7 (scipy)")
plt.ylabel("PC7 (eigenstrat)")

The lower pcs start to slightly deviate. Now compare eigenvalues ...

In [ ]:
lamb_eigenstrat = pd.read_table("./data/freq_fil/data_fil_freqfil.eval", header=None, sep="\t").as_matrix()
pves_eigenstrat = lamb_eigenstrat[:10] / np.sum(lamb_eigenstrat[:10])

lamb_scipy = np.diag(Sigma)**2
pves_scipy = lamb_scipy / np.sum(lamb_scipy)

plt.plot(pves_eigenstrat, 'o', label="eigenstrat")
plt.plot(pves_scipy, 'ro', label="scipy")
plt.xlabel("PC")
plt.ylabel("PVE")
plt.legend(frameon=False)

In [ ]:
plt.scatter(pves_eigenstrat[:,:10], pves_scipy)
plt.xlabel("pve (eigenstrat)")
plt.ylabel("pve (scipy)")